In [1]:
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
from nltk.corpus import stopwords
import re
import os 

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
df = pd.DataFrame()

for json_file in os.listdir('/kaggle/input/massive-dataset-v1-nlpnlu'):
    if json_file.endswith('.jsonl'):
        df = pd.concat([df, pd.read_json('/kaggle/input/massive-dataset-v1-nlpnlu/' + json_file, lines=True)], ignore_index=True)
        print(f"Added {json_file} to dataframe")

Added ko-KR.jsonl to dataframe
Added zh-TW.jsonl to dataframe
Added ru-RU.jsonl to dataframe
Added th-TH.jsonl to dataframe
Added te-IN.jsonl to dataframe
Added am-ET.jsonl to dataframe
Added jv-ID.jsonl to dataframe
Added cy-GB.jsonl to dataframe
Added hi-IN.jsonl to dataframe
Added fi-FI.jsonl to dataframe
Added mn-MN.jsonl to dataframe
Added ur-PK.jsonl to dataframe
Added km-KH.jsonl to dataframe
Added kn-IN.jsonl to dataframe
Added sl-SL.jsonl to dataframe
Added ro-RO.jsonl to dataframe
Added ml-IN.jsonl to dataframe
Added he-IL.jsonl to dataframe
Added en-US.jsonl to dataframe
Added es-ES.jsonl to dataframe
Added zh-CN.jsonl to dataframe
Added da-DK.jsonl to dataframe
Added nl-NL.jsonl to dataframe
Added ar-SA.jsonl to dataframe
Added sv-SE.jsonl to dataframe
Added tl-PH.jsonl to dataframe
Added is-IS.jsonl to dataframe
Added fr-FR.jsonl to dataframe
Added my-MM.jsonl to dataframe
Added nb-NO.jsonl to dataframe
Added id-ID.jsonl to dataframe
Added az-AZ.jsonl to dataframe
Added af

In [4]:
def remove_punctuation(data: pd.DataFrame) -> pd.DataFrame:
    """Remove punctuation from text"""
    data["utt"] = data["utt"].str.replace(r"[^\w\s]","", regex=True)
    return data

def lowercase(data: pd.DataFrame) -> pd.DataFrame:
    """Lowercase text"""
    data["utt"] = data["utt"].str.lower()
    return data

def drop_cols(data: pd.DataFrame) -> pd.DataFrame:
    drop = ["worker_id", "slot_method", "judgments"]
    return data.drop(drop, axis=1)

def encode_labels(data: pd.DataFrame) -> pd.DataFrame:
    """Encode labels"""
    le = LabelEncoder()
    le.fit(data['intent'])
    data['intent'] = le.transform(data['intent'])
    return data, le

def decode_labels(data: np.ndarray, le: LabelEncoder) -> np.ndarray:
    """Decode labels"""
    data = le.inverse_transform(data)
    return data

In [5]:
from toolz.functoolz import pipe

df["locale"] = df["locale"].apply(lambda x: x.split("-")[0])

params = [
    remove_punctuation,
    drop_cols,
    lowercase,
]


df = pipe(
    df,
    *params
)


df, encoder = encode_labels(df)

print(f"Finished preprocessing dataset.\n\n")

Finished preprocessing dataset.




In [6]:
df.head()

,id,locale,partition,scenario,intent,utt,annot_utt
0,0,ko,test,alarm,2,이번 주 오전 다섯 시 에 깨워줘,[date : 이번 주] [time : 오전 다섯 시] 에 깨워줘
1,1,ko,train,alarm,2,금요일 오전 아홉 시 에 깨워줘,[date : 금요일] [time : 오전 아홉 시] 에 깨워줘
2,2,ko,train,alarm,2,지금부터 두시 간 동안 알람 설정해,[time : 지금부터 두시 간] 동안 알람 설정해
3,3,ko,test,audio,4,조용히 해,조용히 해
4,4,ko,train,audio,4,올리 조용히 해,올리 조용히 해


In [7]:
train_df = df.loc[df['partition'] == 'train']
eval_df = df.loc[df['partition'] == 'dev']

In [8]:
utterances = [train_df['utt'].values, eval_df['utt'].values]
num_labels = len(df['intent'].unique())
labels = [train_df['intent'].values, eval_df['intent'].values]

In [9]:
import gc
del df
del train_df
del eval_df
gc.collect()

%reset_selective -f df
%reset_selective -f train_df
%reset_selective -f eval_df

In [16]:
from numpy import save
save('utterances.npy', utterances)
save('labels.npy', labels)

/opt/conda/lib/python3.10/site-packages/numpy/lib/npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [3]:
from numpy import load
utterances = load('utterances.npy', allow_pickle = True)
labels = load('labels.npy', allow_pickle = True)

In [4]:
num_labels = 60

In [5]:
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [8]:
models = ["xlm-roberta-base", "microsoft/mdeberta-v3-base", "google/mt5-small"]

In [6]:
idx = 0
model = None
tokenizer = None

In [9]:
if models[idx] == "google/mt5-small":
    model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
else:
    model = AutoModelForSequenceClassification.from_pretrained(models[1], num_labels=num_labels)
tokenizer = AutoTokenizer.from_pretrained(models[1])

Some weights of the model checkpoint at microsoft/mdeberta-v3-base were not used when initializing DebertaV2ForSequenceClassification: ['deberta.embeddings.word_embeddings._weight', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.bias', 'mask_predictions.dense.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.classifier.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializi

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
model = torch.nn.DataParallel(model)

model = model.to(device)

In [11]:
print(' Original: ', utterances[0][0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(utterances[0][0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(utterances[0][0])))

 Original:  금요일 오전 아홉 시 에 깨워줘
Tokenized:  ['▁', '금', '요', '일', '▁오', '전', '▁아', '홉', '▁시', '▁', '에', '▁', '깨', '워', '줘']
Token IDs:  [260, 3688, 4662, 667, 4559, 3228, 3760, 240699, 6464, 260, 874, 260, 56580, 11110, 159035]


In [12]:
max_len = 210
"""
for sent in utterances:
    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))
"""
print('Max sentence length: ', max_len)

Max sentence length:  210


In [ ]:
input_ids = []
attention_masks = []

for utt_ in utterances:
    input_ids_ = []
    attention_masks_ = []
    for utt in utt_:
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                            utt,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 128,           # Pad & truncate all sentences.
                            truncation = True,
                            padding = 'max_length',
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                       )

        # Add the encoded sentence to the list.    
        input_ids_.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks_.append(encoded_dict['attention_mask'])
    
    input_ids.append(input_ids_)
    attention_masks.append(attention_masks_)

# Convert the lists into tensors.
input_ids = [torch.cat(inp, dim=0) for inp in input_ids]
attention_masks = [torch.cat(att, dim=0) for att in attention_masks]
labels = [torch.tensor(lab) for lab in labels]

# Print sentence 0, now as a list of IDs.
print('Original: ', utterances[0][0])
print('Token IDs:', input_ids[0][0])

In [ ]:
train_dataset = TensorDataset(input_ids[0], attention_masks[0], labels[0])
val_dataset = TensorDataset(input_ids[1], attention_masks[1], labels[1])

In [ ]:
# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 2

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
criterion = nn.CrossEntropyLoss().cuda()

In [ ]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()
# For each epoch...
for epoch_i in range(0, epochs):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Measure how long the training epoch takes.
    t0 = time.time()
    total_train_loss = 0
    model.train()
    batches = tqdm(enumerate(train_dataloader), desc=f"Epoch {epoch_i + 1}/{epochs}", total=len(train_dataloader))
    for step, batch in batches:
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the device using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        output = model(b_input_ids, 
                             #token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)        
        
        loss = criterion(output.logits, b_labels)
        total_train_loss += loss.item()
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    # Tracking variables 
    total_eval_accuracy = 0
    best_val_loss = 99999
    total_eval_loss = 0
    nb_eval_steps = 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        
            output= model(b_input_ids, 
                                   #token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
        loss = criterion(output.logits, b_labels)
        total_eval_loss += loss.item()
        # Move logits and labels to CPU if we are using GPU
        logits = output.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    print("  Loss: {0:.2f}".format(avg_val_loss))
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    if avg_val_loss < best_val_loss:
        torch.save(model, 'mdeberta' + "-MASSIVE-finetuned")
        best_val_loss = avg_val_loss
    # print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    # print("  Validation took: {:}".format(validation_time))
    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [23]:
training_stats

[{'epoch': 1,
  'Training Loss': 0.8373585505814293,
  'Valid. Loss': 0.7208714448498887,
  'Valid. Accur.': 0.8298461123110151,
  'Training Time': '3:48:51',
  'Validation Time': '0:13:21'},
 {'epoch': 2,
  'Training Loss': 0.3526039081991805,
  'Valid. Loss': 0.7140008347985111,
  'Valid. Accur.': 0.843952483801296,
  'Training Time': '3:49:01',
  'Validation Time': '0:13:28'}]